In [16]:
import pandas as pd
import statistics
import datetime 
import matplotlib.pyplot as plt

In [64]:
# 計算夏普比率
def calculate_sharpe(R,rf,sd_s): # 基金持有至目前報酬率 & 大盤報酬率 & 基金淨值標準差 & 同類型大盤標準差
    sharpe = (R-rf)/sd_s
    return sharpe 
# 計算報酬率
def sell_return(today_nv, buy_nv, today_date, buy_date): # 計算賣出報酬率
    hold_days = (today_date - buy_date).days
    ret = (today_nv - buy_nv)/ buy_nv

    return ret
def down_days(s_list):
    day = 0
    for i in range(len(s_list)):
        if(i>0):
            if s_list[i] < s_list[i-1]:
                day += 1
            else:
                break
    return day
    

In [73]:
# import data #
#基金資料, 多 read 幾個sheet包成迴圈就可以一次測很多檔基金，可以把結果輸出成excel
df_fund = pd.read_excel("基金資料.xlsx",sheet_name ="工作表4") 
df_fund['年月日'] = pd.to_datetime(df_fund['年月日'])
df_market = pd.read_excel("大盤資料.xlsx") #大盤資料，自行指定大盤指數

df_fund

,年月日,幣別,淨值(元),基金名稱
0,2013-11-26,NTD,9.9998,野村環高債A
1,2013-11-27,NTD,9.9994,NaN
2,2013-11-28,NTD,9.9989,NaN
3,2013-11-29,NTD,9.9985,NaN
4,2013-12-02,NTD,9.9971,NaN
5,2013-12-03,NTD,10.0030,NaN
6,2013-12-04,NTD,10.0027,NaN
7,2013-12-05,NTD,10.0015,NaN
8,2013-12-06,NTD,10.0019,NaN
9,2013-12-09,NTD,10.0075,NaN


In [75]:
#低檔投資法回測 ：10% 加碼, 淨值若比10天前下跌10%,「隔天」就加碼
df_10_percent = pd.DataFrame(columns = ["買進日","買入價格","加碼"]) # 存回測結果
unit = 1 # 持有單位,假設創立之初先買一單位
start_index = 0 # 測試不同時間開始的報酬率
buy_nv = df_fund.loc[0,'淨值(元)'] # 假設成立當日就先買一單位
df_10_percent = df_10_percent.append({"買進日":df_fund.loc[0,'年月日'],
                                      "買入價格" : buy_nv, 
                                      "加碼": "x"}, ignore_index=True)

for i,item in df_fund.iterrows():
    if((item['年月日']-df_fund.loc[start_index,'年月日']).days >= 30): # 定期定額
        unit = unit + 1
        buy_nv = buy_nv + df_fund.loc[i,'淨值(元)']
        start_index = i
        df_10_percent = df_10_percent.append({"買進日": item['年月日'], 
                                              "買入價格" : item['淨值(元)'], 
                                              "加碼": "x"}, ignore_index=True)
    
    if(i>=10):
        if( item['淨值(元)'] <= 0.9*(df_fund.loc[i-10,'淨值(元)']) ): # 加碼
            buy_date = df_fund.loc[i+1,'年月日']
            unit = unit + 1
            buy_nv = buy_nv + df_fund.loc[i+1,'淨值(元)']
            df_10_percent = df_10_percent.append({"買進日":buy_date, 
                                                  "買入價格" : df_fund.loc[i+1,'淨值(元)'], 
                                                  "加碼": "v"}, ignore_index=True)
print("end loop")
print(unit*df_fund.loc[len(df_fund)-1,'淨值(元)'])
print(buy_nv)
profit_ret = sell_return(unit*df_fund.loc[len(df_fund)-1,'淨值(元)'], buy_nv,
                         df_fund.loc[len(df_fund)-1,'年月日'], df_10_percent.loc[0,'買進日'])
print(profit_ret)   
#pd.set_option('display.max_rows', None)
df_10_percent

end loop
837.9895
860.9841000000001
-0.026707345698950887


,買進日,買入價格,加碼
0,2013-11-26,9.9998,x
1,2013-12-26,10.0924,x
2,2014-01-27,10.1815,x
3,2014-02-26,10.2918,x
4,2014-03-28,10.3419,x
5,2014-04-28,10.3596,x
6,2014-05-28,10.4042,x
7,2014-06-27,10.4543,x
8,2014-07-28,10.3821,x
9,2014-08-27,10.4033,x


In [76]:
# Sharpe 投資法

df_sharpe = pd.DataFrame(columns = ["買進日","買入價格","加碼","Sharpe"]) # 存回測結果
unit = 1 # 持有單位,假設創立之初先買一單位
start_index = 0 # 測試不同時間開始的報酬率
buy_nv = df_fund.loc[0,'淨值(元)'] # 假設成立當日就先買一單位
nv_list = list(df_fund['淨值(元)'])
df_sharpe = df_sharpe.append({"買進日":df_fund.loc[0,'年月日'],
                                      "買入價格" : buy_nv, 
                                      "加碼": "x"}, ignore_index=True)

sharpe_list = []
for i,item in df_fund.iterrows():
    rf = df_market.loc[i,"1M"]/100
    if i > 2: # 第2天後每天計算 sharpe
        R = sell_return(unit*item['淨值(元)'], buy_nv, item['年月日'], df_sharpe.loc[0,'買進日'])
        l = nv_list[0:i]
        sd_s =  statistics.stdev(l)
        shar = calculate_sharpe(R,rf,sd_s)
        sharpe_list.append(shar) # list 的 index 和 df 差 10 => 11 vs 0

    if((item['年月日']-df_fund.loc[start_index,'年月日']).days >= 30): # 定期定額
        unit = unit + 1
        buy_nv = buy_nv + df_fund.loc[i,'淨值(元)']
        start_index = i
        df_sharpe = df_sharpe.append({"買進日": item['年月日'], 
                                      "買入價格" : item['淨值(元)'], 
                                      "加碼": "x",
                                      "Sharpe" : shar }, ignore_index=True)
    if i > 12:
        if ( (1 + sharpe_list[len(sharpe_list)-1]) < 0.9*(1 + sharpe_list[len(sharpe_list)-11]) ):  # 加碼            
            tmp = sharpe_list[len(sharpe_list)-4:]
            count_down = down_days(tmp) # 計算夏普指數連跌幾天
            #if count_down >= 3:    
            buy_date = df_fund.loc[i+1,'年月日']
            unit = unit + 1
            buy_nv = buy_nv + df_fund.loc[i+1,'淨值(元)']
            df_sharpe = df_sharpe.append({"買進日": buy_date, 
                                          "買入價格" : df_fund.loc[i+1,'淨值(元)'], 
                                          "加碼": "v",
                                          "Sharpe" : shar }, ignore_index=True)
print(sell_return(unit*df_fund.loc[len(df_fund)-1,'淨值(元)'], buy_nv,
                         df_fund.loc[len(df_fund)-1,'年月日'], df_sharpe.loc[0,'買進日']))
df_sharpe

-0.01628632888053362


,買進日,買入價格,加碼,Sharpe
0,2013-11-26,9.9998,x,NaN
1,2013-12-26,10.0924,x,-0.186099
2,2014-01-27,10.1815,x,-0.012310
3,2014-02-06,10.1527,v,-0.126939
4,2014-02-07,10.1832,v,-0.126150
5,2014-02-10,10.1889,v,-0.101308
6,2014-02-11,10.2098,v,-0.109386
7,2014-02-12,10.2151,v,-0.097923
8,2014-02-13,10.2348,v,-0.100443
9,2014-02-26,10.2918,x,-0.011190
